# Eurostat European Big Data Hackathon 2025

<img src="https://cros.ec.europa.eu/system/files/inline-images/EU-Big-Data-Hackathon-2025_horiz_0.jpg" width="500"/>


Eurostat is organizing the fifth round of the European Big Data Hackathon from 6 to 11 March 2025 (including the presentation by the winners at the NTTS) in Brussels. 

The purpose of the 2025 hackathon is to foster expertise in integrating Earth Observation data with official statistics for producing innovative ideas for statistical products and tools relevant for the EU policies. 


The European Big Data Hackathon takes place every two years and gathers teams from all over Europe to compete for the best solution to a statistical challenge. The teams develop innovative approaches, applications and data products combining official statistics and big data that can help to answer pressing EU policy and/or statistical questions.

_Source: https://cros.ec.europa.eu/2025EuropeanBigDataHackathon_

# How to download, visualise and run some basic statistics on ERA5 data

*Written by William Ray for the participants of the 5th European Big Data Hackathon 2025.*

In this notebook you are shown how to:

- Download ERA5 data from the Climate Data Store using API.
- How to visualise the downloaded data using xarray.
- Run some basic statistics such as the average temperature over a year.
- Visualise the average temperature difference between 1950 and 2023.

Before you get started you will need to create an account at the [Climate Data Store](https://cds.climate.copernicus.eu/).



In [17]:
import sys
!{sys.executable} -m pip install 

#cdsapi cartopy basemap-data-hires 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 MB 42.9 MB/s eta 0:00:0000:0100:01


In [18]:
#!pip install xarray dask cartopy cdsapi netCDF4
# Python Standard Libraries
import os
import datetime as dt

# Data Manipulation Libraries
import numpy as np
import xarray as xr

# Visualization Libraries
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# Climate Data Store API for retrieving climate data
import cdsapi

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Downloading the Data

First, we'll load ERA5 data from the [Climate Data Store (CDS)](https://doi.org/10.24381/cds.f17050d7) using the `cdsapi`, including the land-sea mask. To do this, provide your CDS API key.

In [ ]:
URL = 'https://cds.climate.copernicus.eu/api/'
KEY = 'APIKEY' # add your key here the format should be as {uid}:{api-key}

New to CDS? Consider reading the [CDS tutorial](https://ecmwf-projects.github.io/copernicus-training-c3s/cds-tutorial.html) for a detailed guide.


In [ ]:
file_name = {}  # dictionary containing [data source : file name]

# Add the data sources and file names
file_name.update({"era5": "temperature_era5.nc"})

# Create the paths to the files
path_to = {
    source: os.path.join(f"data/{source}/", file) for source, file in file_name.items()
}

# Create necessary directories if they do not exist
for path in path_to.values():
    os.makedirs(
        os.path.dirname(path), exist_ok=True
    )  # create the folder if not available

path_to

In [ ]:
c = cdsapi.Client(url=URL, key=KEY)

c.retrieve(
  'reanalysis-era5-single-levels-monthly-means',
  {
    'format': 'netcdf',
    'product_type': 'monthly_averaged_reanalysis',
    'variable': ['2m_temperature', 'land_sea_mask'],
    'year': list(range(1950, 2023)),
    'month': list(range(1, 13)),
    'time': '00:00',
    # 'grid': [0.25, 0.25],
  },
  path_to['era5']
)

## Opening the dataset

Now it is downloaded, we can open the dataset and inspect it using `xarray`.

In [ ]:
data = xr.open_dataset('data/era5/temperature_era5.nc')
data

In [ ]:
lat = data.latitude
lon = data.longitude
longitude= data.longitude-180
data = data.sortby(longitude)

temp_2m = data.t2m[0,:,:]


In [ ]:
m = Basemap(projection='cyl', llcrnrlon=-20, llcrnrlat=30, urcrnrlon=50, urcrnrlat=80,resolution='i')
m.drawcoastlines(1)
m.drawcountries()

parallels = np.arange(-90,90+0.25,20)
m.drawparallels(parallels, labels=[1,0,0,0], linewidth=0.5)
meridians = np.arange(-180,180+0.25,20)
m.drawmeridians(meridians, labels=[0,0,0,1], linewidth=0.5)

cf = plt.contourf(longitude, lat, temp_2m, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.02)
cb.set_label(' \u00b0K', fontsize=15, rotation=0)

plt.show()

## Calculate the yearly average for 1950

In [ ]:
start_date = "1950-01-01"
end_date = "1950-12-01"

temp_1950 = data.sel(valid_time=slice(start_date, end_date))
temp_1950

In [ ]:
mean_1950 = temp_1950.t2m.sum(dim="valid_time") / 12
mean_1950

In [ ]:
m = Basemap(projection='cyl', llcrnrlon=-20, llcrnrlat=30, urcrnrlon=50, urcrnrlat=80,resolution='i')
m.drawcoastlines(1)
m.drawcountries()

parallels = np.arange(-90,90+0.25,20)
m.drawparallels(parallels, labels=[1,0,0,0], linewidth=0.5)
meridians = np.arange(-180,180+0.25,20)
m.drawmeridians(meridians, labels=[0,0,0,1], linewidth=0.5)

cf = plt.contourf(longitude, lat, mean_1950, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.02)
cb.set_label(' \u00b0K', fontsize=15, rotation=0)

plt.show()

## Calculate the yearly average for 2020 and then visualise the difference between the average temperature in 1950 and 2020

In [ ]:
start_date = "2020-01-01"
end_date = "2020-12-01"

temp_2020 = data.sel(valid_time=slice(start_date, end_date))
temp_2020

In [ ]:
mean_2020 = temp_2020.t2m.sum(dim="valid_time") / 12
mean_2020

In [ ]:
m = Basemap(projection='cyl', llcrnrlon=-20, llcrnrlat=30, urcrnrlon=50, urcrnrlat=80,resolution='i')
m.drawcoastlines(1)
m.drawcountries()

parallels = np.arange(-90,90+0.25,20)
m.drawparallels(parallels, labels=[1,0,0,0], linewidth=0.5)
meridians = np.arange(-180,180+0.25,20)
m.drawmeridians(meridians, labels=[0,0,0,1], linewidth=0.5)

cf = plt.contourf(longitude, lat, mean_1950 - mean_2020, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.02)
cb.set_label(' \u00b0K', fontsize=15, rotation=0)

plt.show()